# Example usage of the Regression package using the California Housing Dataset

### Importing the dataset and libraries

In [1]:
from sklearn.datasets import fetch_california_housing
import numpy as np

In [2]:
# Loading the data
california_housing = fetch_california_housing()
X = california_housing.data # Covariates
y = california_housing.target # Response

## OLS Example

In [3]:
# Import the OLS class
from regression.estimators import OLS

### Basic calculations

**Fitting the OLS model**

In [4]:
# Initialize the OLS model with default parameters
ols_model = OLS()

# Regress y on x
ols_model.fit(X, y)
ols_model.beta # The OLS estimator

array([ 2.06855817,  0.8296193 ,  0.11875165, -0.26552688,  0.30569623,
       -0.004503  , -0.03932627, -0.89988565, -0.870541  ])

**Predicting using the fitted model**

In [5]:
ols_model.predict(X)
# Note: Normally, one would use the fitted model to predict unseen data, not the same X used when fitting.

array([4.13164983, 3.97660644, 3.67657094, ..., 0.17125141, 0.31910524,
       0.51580363])

**Compute performance metrics**

In [6]:
R2 = ols_model.score(y) # Normal R^2
R2_ajd = ols_model.score(y, adjusted=True) # Adjusted R^2
MSE = ols_model.mse(y) # Uses the values of y_hat store in the model when fitting

print(
    f"R2: {R2}\n"
    f"Adjusted R2: {R2_ajd}\n"
    f"MSE: {MSE}"
)

R2: 0.6062326851998039
Adjusted R2: 0.6060609094335685
MSE: 0.5243209861846072


### Testing with the OLS model

**T-Test for a single coefficient**

In [7]:
j = 3
bj = 1.3
alpha = 0.05
ols_model.t_test(j, bj, alpha) # Using critical value

With significance level 5.0%, we REJECT H0.


False

In [8]:
ols_model.t_test(j, bj, alpha, use_p_value=True) # Using p value

With significance level 5.0%, we REJECT H0.


False

**Testing for a model with reduced covariates**

In [9]:
reduced_X = X[:, :5] # Reduced model with only 5 covariates
alpha = 0.05
ols_model.f_test_reduced_model(reduced_X, y, alpha)

With significance level 5.0%, we REJECT H0, i.e., the full model is correct.


False

**Testing for arbitrary number of constraints**

In [10]:
R = np.array([
    [0, 0, 0, 1, 0, -1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0]
])
r = np.array([0, 0])
# We are testing if beta_3 = beta_5 and if beta_4 = 0.

alpha = 0.05

ols_model.f_test_constraints(R, r, alpha)

With significance level 5.0%, we REJECT H0, i.e., the constraints do NOT hold for this model.


False

### Confidence Intervals with the OLS model

**Confidence interval for a single coefficient**

In [11]:
j = 3 # Index of coefficient
interval1 = ols_model.confidence_interval_single(j, alpha=0.05)

The interval [-0.2940677748709666, -0.23698598414228] has EXACT coverage probability 95.0% for coefficient B_3.


**Bonferonni confidence intervals for multiple coefficients**

In [12]:
test_coefficients = np.array([0, 1, 2, 3]) # Index of coefficients
region1 = ols_model.confidence_interval_bonferonni(test_coefficients, alpha=0.025)

The region defined by
[(2.0547721085601807, 2.0823442296180956),
 (0.8078159859216998, 0.851422622639205),
 (0.10339238272347281, 0.13411091970081232),
 (-0.3053463722050296, -0.225707386808217)]
has coverage probability GREATER THAN 97.5% for coefficients [0 1 2 3].


**Ellipsoid confidence region and test for $\beta$**

In [13]:
test_beta = np.array([2, 1, 0.1, -0.25, 0.3, 0, 0, -1, -1])
ols_model.confidence_ellipsoid_test(test_beta, alpha=0.05)

The 95.0% confidence region for Beta is an ellipsoid centered at
[ 2.06855817  0.8296193   0.11875165 -0.26552688  0.30569623 -0.004503
 -0.03932627 -0.89988565 -0.870541  ]
with radius 16.923045113096883.
This region DOES NOT contain your particular Beta = 
[ 2.    1.    0.1  -0.25  0.3   0.    0.   -1.   -1.  ].


False

**Confidence interval for $m(x_{new})$ and $y_{new}$**

In [14]:
x_new = np.array([0, 5, 40, 5, 1, 300, 2, 37, -120])
interval_m = ols_model.confidence_interval_predicted(x_new, alpha = 0.05) # For m(x_new)

The interval [73.61740011134825, 81.61424497138975] has EXACT coverage probability 95.0% for m(x_new).


In [15]:
interval_y = ols_model.confidence_interval_predicted(x_new, alpha = 0.05, for_y=True) # For y_new

The interval [73.37286902882846, 81.85877605390954] has EXACT coverage probability 95.0% for y_new.


## WLS Example

In [16]:
# Import the WLS class
from regression.estimators import WLS

### Basic Calculations

**Fitting the WLS models using specified weights**

In [17]:
# Initialize the WLS model with default parameters
wls_model = WLS()

weights = np.random.normal(size=len(X)) # N weights
# Regress y on x
wls_model.fit(X, y, weights)
wls_model.beta # The WLS estimator

array([ 3.01112026,  0.94568743,  1.15787419,  0.52357691, -0.79419503,
       -0.02934525, -0.2173963 ,  6.63703316,  4.97191875])

**Predicting using the fitted model**

In [18]:
wls_model.predict(X)
# Note: Normally, one would use the fitted model to predict unseen data, not the same X used when fitting.

array([7.24023211, 5.2356188 , 7.78578164, ..., 8.50641495, 8.38399287,
       8.44490132])

**Compute performance metrics**

In [19]:
R2 = wls_model.score(y) # Normal R^2
R2_ajd = wls_model.score(y, adjusted=True) # Adjusted R^2
MSE = wls_model.mse(y) # Uses the values of y_hat store in the model when fitting

print(
    f"R2: {R2}\n"
    f"Adjusted R2: {R2_ajd}\n"
    f"MSE: {MSE}"
)

R2: 6.587853075058999
Adjusted R2: 6.590290701818513
MSE: 10.772439615745995


### Feasible WLS

**A pratical way to do WLS estimation using OLS**

In [20]:
wls_model.feasible_wls(X, y)
wls_model.beta

array([ 2.06855817,  1.98192896, -0.26295792, -2.44315985,  1.92891399,
       -0.36089154, -0.16107315,  2.97487941,  2.338146  ])

In [21]:
# Calculate AIC
aic = ols_model.compute_aic(X, y)
aic

45263.541611265886

In [22]:
# Calculate BIC
bic = ols_model.compute_bic(X, y)
bic

45327.021501022646

## Ridge Example

In [23]:
# Import the Ridge class
from regression.estimators import Ridge

In [24]:
# Initialize the Ridge model with default parameters
ridge_model = Ridge()

# Regress y on x
ridge_model.fit(X, y, lambdaa = 0.1) # Set a lambda value > 0.
ridge_model.beta # The Ridge estimator

array([ 2.06854815,  0.82961664,  0.11875818, -0.26551388,  0.30567906,
       -0.00450071, -0.03932662, -0.89982369, -0.87047846])

Other basic calculations are done in the same way as with OLS and WLS.

## ANOVA Example

In [27]:
# Import the ANOVA class
from regression.anova import ANOVA

**Generating some fake data**

In [40]:
# Some fake data
group_size = 50
group1 = np.random.normal(loc=10, scale=2, size=group_size)
group2 = np.random.normal(loc=12, scale=2, size=group_size)
group3 = np.random.normal(loc=15, scale=2, size=group_size)

values = np.concatenate([group1, group2, group3])
groups = np.array(['1'] * group_size + ['2'] * group_size + ['3'] * group_size)

# Shuffle the data
shuffled_indices = np.random.permutation(len(values))
values = values[shuffled_indices]
groups = groups[shuffled_indices]

**Mean computation**

In [41]:
# Initialize the model with data
anova_model = ANOVA(groups, values)

# Compute the means
anova_model.compute_means()

array([ 9.96047747, 11.92077677, 14.82937052])

**Sum of squares computation**

In [42]:
anova_model.compute_ss()
print(
    f"TSS: {anova_model.tss}\n"
    f"WSS: {anova_model.wss}\n"
    f"BSS: {anova_model.bss}"
)

TSS: 1129.43089174838
WSS: 529.2840512858883
BSS: 600.1468404624917


**Conducting an ANOVA F-test**

In [43]:
anova_model.f_test(alpha = 0.05)

With significance level 5.0%, we REJECT H0, i.e., there exists a group with different mean.


True

**Outputting an ANOVA table**

In [44]:
info = anova_model.table()

+----------+-----+-------------------+--------------------+-------------------+------------------------+
|  Effect  |  DF |     Effect SS     |     Effect MSE     |       F-Stat      |        P-Value         |
+----------+-----+-------------------+--------------------+-------------------+------------------------+
|  Factor  |  2  | 600.1468404624917 | 300.07342023124585 | 83.34049111592647 | 1.1102230246251565e-16 |
| Residual | 147 |  1129.43089174838 | 7.683203345227074  |                   |                        |
+----------+-----+-------------------+--------------------+-------------------+------------------------+
